In [27]:
%load_ext autoreload
%autoreload 2

import gtab
import matplotlib.pyplot as plt
import gensim.downloader
from gensim.similarities.fastss import FastSS
import numpy as np

from get_related_words import get_hypernyms
from get_related_words import get_hyponyms
from get_related_words import get_synonyms
from get_related_words import get_wordnet_path_similarity
from get_related_words import get_similar_words

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [21]:
TERM = 'Ice Cream'

In [24]:
#terms = get_similar_words(TERM, n_words=50)
get_similar_words(TERM, n_words=50)

Loading saved model...
[('icecreams', 0.8418611884117126), ('ice-cream', 0.8290021419525146), ('icecream', 0.7736142873764038), ('ice-creams', 0.7453774809837341), ('ice-cream', 0.7384599447250366), ('icecube', 0.7137746214866638), ('ice-', 0.6846408247947693), ('ice', 0.6809401512145996), ('ice-box', 0.6777350902557373), ('ice-cube', 0.6772070527076721), ('ice-water', 0.6720316410064697), ('ice-pack', 0.6456763744354248), ('slushie', 0.6299453377723694), ('ice-berg', 0.6293495893478394), ('ice-rink', 0.6291958093643188), ('popsicle', 0.6275215744972229), ('ice-melt', 0.6272243857383728), ('ice-bucket', 0.6266813278198242), ('ice-skate', 0.6247037649154663), ('ice-cubes', 0.6243662238121033), ('ices', 0.6242417097091675), ('milkshake', 0.6235678195953369), ('sundaes', 0.6233888268470764), ('creamer', 0.6230928301811218), ('iceman', 0.6225195527076721), ('icey', 0.621169924736023), ('icemaker', 0.6197085380554199), ('ice-cap', 0.6183856725692749), ('ice-encrusted', 0.6178804636001587), 

{'icecream': ['ice-laden',
  'hotdog',
  'ice-cold',
  'chocolate',
  'frappuccino',
  'flapjack',
  'ice-',
  'chocolate-chip',
  'pizza',
  'cloudberry',
  'ice-bucket',
  'ice-filled',
  'piehole',
  'teacake',
  'marmite',
  'lolly',
  'yoghurt',
  'ice-cap',
  'vego',
  'bilberry',
  'ice-fishing',
  'snowjob',
  'ice-nine',
  'iceball',
  'soda',
  'ice-encrusted',
  'sherbet',
  'budweiser',
  'liquorice',
  'pavlovas',
  'ice-out',
  'snowmachine',
  'fro-yo',
  'mcflurry',
  'tea-tree',
  'orangeade',
  'ice-breaker',
  'ice-skate',
  'ice-time',
  'cola',
  'dairy-free',
  'liquer',
  'glacier',
  'ice-cover',
  'doughnut',
  'melt-water',
  'ice-melting',
  'ice-class',
  'slurpee',
  'snowglobe']}

In [53]:
path_similarities = [get_wordnet_path_similarity(TERM, t) for t in terms[TERM]]

In [ ]:
sorted(list(zip(path_similarities, terms[TERM])), reverse=True)

[(1.0, 'coffee'), (0.5, 'espresso'), (0.5, 'decaf'), (0.5, 'cappuccinos'), (0.5, 'cappuccino'), (0.3333333333333333, 'tea'), (0.3333333333333333, 'cocoa'), (0.3333333333333333, 'chocolate'), (0.25, 'cuppers'), (0.25, 'cupper'), (0.25, 'cuppas'), (0.25, 'cuppa'), (0.2, 'beer'), (0.09090909090909091, 'decaffeinated'), (0.09090909090909091, 'beans'), (0.08333333333333333, 'roasters'), (0.08333333333333333, 'roaster'), (0.07692307692307693, 'coffeehouse'), (0.07692307692307693, 'cafe'), (0.07142857142857142, 'creamer'), (0.07142857142857142, 'coffeepot'), (0, 'tea-'), (0, 'starbucks'), (0, 'softdrinks'), (0, 'roastery'), (0, 'non-coffee'), (0, 'fruit-juice'), (0, 'frappucino'), (0, 'frappuccinos'), (0, 'frappuccino'), (0, 'frappuccino'), (0, 'decaffeination'), (0, 'decaff'), (0, 'coffeeshops'), (0, 'coffeeshop'), (0, 'coffeemaker'), (0, 'coffee-shop'), (0, 'coffee-pot'), (0, 'coffee-making'), (0, 'coffee-maker'), (0, 'coffee-house'), (0, 'coffee-flavoured'), (0, 'coffee-flavored'), (0, 'co

In [29]:
hypernyms = get_hypernyms('icecream')
print('hypernyms: {}'.format(hypernyms))

hypernyms: frozen_dessert


In [30]:
synonyms = get_synonyms('icecream')
print('synonyms: {}'.format(synonyms))

synonyms: [['ice_cream']]


In [40]:
hyponyms = get_hyponyms('frozen_dessert')
print('synonyms: {}'.format(hyponyms))

synonyms: [['frozen_pudding'], ['ice-cream_cake', 'icebox_cake'], ['frozen_custard', 'soft_ice_cream'], ['ice_lolly', 'lolly', 'lollipop', 'popsicle'], ['ice_milk'], ['split'], ['snowball'], ['ice_cream', 'icecream'], ['ice-cream_sundae', 'sundae'], ['ice', 'frappe'], ['frozen_yogurt'], ['parfait'], ['snowball'], ['ice-cream_cone'], ['sherbert', 'sherbet']]


In [47]:
get_wordnet_path_similarity(TERM, 'tea')

0.3333333333333333

In [ ]:
from nltk.corpus import wordnet as wn
print(wn.synsets('icecream'))

[Synset('ice_cream.n.01')]
